# Paramétrage

## Librairies


In [ ]:
import numpy as np
import glob
import plotly.express as px
import plotly.graph_objs as go
import cv2
import pandas as pd
import geopandas as gpd
import tqdm
from datetime import date
from google.colab import drive
import random
import matplotlib.pyplot as plt
import skimage as ski
import shapely
import os
import math


# Statistiques

In [ ]:
def calc_F1(precision, recall):
  if precision + recall == 0:
    return 0
  else:
    return 2*precision*recall/(precision + recall)


## Semantic segmentation

In [ ]:
color_discrete_map = {
    'Unet_binary_20240106' : '#e7298a',
    'UNet07_res512_23_12_23' : '#c994c7',
    'DeeplabV3Plus_20240107' : '#74add1',
    'DeeplabV3Plus_3_20240112' : '#abd9e9',
    'SegFormer' : '#de77ae',
    'YOLOv8-seg' : '#bf812d',
    'YOLOv8-seg (Bretagne)' : '#35978f'
    }


In [ ]:
list_files = [
    '/content/drive/MyDrive/DataScientest/Keras/Unet_binary_metriques_seuil_20240106.csv',
    '/content/drive/MyDrive/DataScientest/Keras/Moez-UNet07_res512_23_12_23_metriques_seuil.csv',
    '/content/drive/MyDrive/DataScientest/Keras/DeeplabV3Plus_metriques_seuil_20240107.csv',
    '/content/drive/MyDrive/DataScientest/Keras/DeeplabV3Plus_3_metriques_seuil_20240112.csv',
    '/content/drive/MyDrive/DataScientest/Keras/SegFormer_metriques_seuil_20240118.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_metriques_seuil_20240124.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_metriques_seuil_20240124-alt.csv'
]
list_modeles = [
    'Unet_binary_20240106',
    'UNet07_res512_23_12_23',
    'DeeplabV3Plus_20240107',
    'DeeplabV3Plus_3_20240112',
    'SegFormer',
    'YOLOv8-seg',
    'YOLOv8-seg (Bretagne)'
]
list_df_iou = []
list_df_PR = []

for file, modele in zip(list_files, list_modeles):
  df = pd.read_csv(file)
  list_df_iou.append(
      df.groupby(['seuil', 'metrique']).agg({'valeur':'mean'}).reset_index()\
      .query("metrique in ['IoU', 'precision']")\
      .assign(modele = modele))
  list_df_PR.append(
      df.groupby(['seuil', 'metrique']).agg({'valeur':'mean'}).reset_index()\
      .query("metrique in ['precision', 'recall']")\
      .pivot(index = ['seuil'], columns = ['metrique'], values = 'valeur').reset_index()\
      .assign(modele = modele))

In [ ]:
df_IoU_max = pd.concat(list_df_iou).reset_index()
df_IoU_max.sort_values(['modele', 'valeur']).groupby(['modele']).tail(1)

,index,seuil,metrique,valeur,modele
125,143,1.0,precision,1.00000,DeeplabV3Plus_20240107
167,143,1.0,precision,1.00000,DeeplabV3Plus_3_20240112
209,143,1.0,precision,0.96548,SegFormer
83,143,1.0,precision,1.00000,UNet07_res512_23_12_23
41,143,1.0,precision,1.00000,Unet_binary_20240106
251,143,1.0,precision,1.00000,YOLOv8-seg
293,143,1.0,precision,1.00000,YOLOv8-seg (Bretagne)


In [ ]:
fig_seuils = px.line(
    pd.concat(list_df_iou),
    x = 'seuil',
    y = 'valeur',
    color = 'modele',
    color_discrete_map=color_discrete_map)
fig_seuils.update_layout(
    title = 'IoU en fonction du seuil',
    plot_bgcolor='white',
    height = 600,
    width = 600)
fig_seuils.write_html('/content/drive/MyDrive/DataScientest/Keras/IoU_seuils.html')
fig_seuils.show()

In [ ]:
df_PR = pd.concat(list_df_PR).reset_index()
df_PR['distance_ideal'] = np.sqrt((1 - df_PR['precision'])**2 + (1 - df_PR['recall'])**2)
df_PR.sort_values(['modele', 'distance_ideal']).groupby(['modele']).head(1)

metrique,index,seuil,precision,recall,modele,distance_ideal
48,6,0.30,0.842600,0.819598,DeeplabV3Plus_20240107,0.239415
71,8,0.40,0.855799,0.794714,DeeplabV3Plus_3_20240112,0.250871
84,0,0.00,0.928242,0.709376,SegFormer,0.299352
29,8,0.40,0.846863,0.810576,UNet07_res512_23_12_23,0.243582
7,7,0.35,0.767508,0.711040,Unet_binary_20240106,0.370878
107,2,0.10,0.894312,0.754324,YOLOv8-seg,0.267445
127,1,0.05,0.750299,0.714913,YOLOv8-seg (Bretagne),0.378979


In [ ]:
fig_PR = px.line(
    pd.concat(list_df_PR),
    x = 'recall',
    y = 'precision',
    color = 'modele',
    color_discrete_map=color_discrete_map,
    hover_data=['seuil'])
fig_PR.update_layout(
    title = 'Courbe PR',
    plot_bgcolor='white',
    height = 600,
    width = 600)
fig_PR.write_html('/content/drive/MyDrive/DataScientest/Keras/PR_seuils.html')
fig_PR.show()

### COCO benchmark

Calcul avec
IoU = TP/(TP + FP + FN)

In [ ]:
def calcul_IoU_COCO(TP, FP, FN):
  return TP/(TP + FP + FN)

In [ ]:
color_discrete_map = {
    'UNet07_res512_23_12_23' : '#c994c7',
    'YOLOv8-seg' : '#bf812d'}


In [ ]:
list_files = [
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_metriques_seuil_20240413.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_metriques_seuil_20240413.csv'
]
list_modeles = [
    'UNet07_res512_23_12_23',
    'YOLOv8-seg']

list_df_PR_iou = []

for file, modele in zip(list_files, list_modeles):
  df = pd.read_csv(file, index_col = None)
  list_df_PR_iou.append(df.assign(modele = modele))

df_PR_iou = pd.concat(list_df_PR_iou).reset_index()

In [ ]:
df_PR_iou = df_PR_iou.pivot_table(
    values='valeur', index=['i_img', 'modele', 'seuil'], columns='metrique', aggfunc="sum").reset_index()
df_PR_iou['F1'] = df_PR_iou.apply(lambda row: calc_F1(row['precision'], row['recall']), axis = 1)
df_PR_iou.head(20)

metrique,i_img,modele,seuil,FN,FP,IoU,IoU_Keras,TP,formes,precision,recall,sca,scce,F1
0,0,UNet07_res512_23_12_23,0.00,10766.0,39940.0,0.562947,0.538908,65311.0,43.0,0.620526,0.858485,0.709789,0.0,0.720363
1,0,UNet07_res512_23_12_23,0.05,13154.0,16163.0,0.682170,0.516954,62923.0,43.0,0.795628,0.827096,0.709789,0.0,0.811057
2,0,UNet07_res512_23_12_23,0.10,14132.0,15002.0,0.680127,0.507215,61945.0,43.0,0.805035,0.814241,0.709789,0.0,0.809612
3,0,UNet07_res512_23_12_23,0.15,14397.0,2952.0,0.780476,0.503829,61680.0,43.0,0.954326,0.810758,0.709789,0.0,0.876703
4,0,UNet07_res512_23_12_23,0.20,14854.0,2952.0,0.774693,0.498116,61223.0,43.0,0.954001,0.804750,0.709789,0.0,0.873043
5,0,UNet07_res512_23_12_23,0.25,16827.0,2797.0,0.751201,0.476243,59250.0,43.0,0.954921,0.778816,0.709789,0.0,0.857925
6,0,UNet07_res512_23_12_23,0.30,16958.0,2796.0,0.749550,0.474959,59119.0,43.0,0.954841,0.777094,0.709789,0.0,0.856847
7,0,UNet07_res512_23_12_23,0.35,24057.0,2481.0,0.662190,0.406881,52020.0,43.0,0.954478,0.683781,0.709789,0.0,0.796765
8,0,UNet07_res512_23_12_23,0.40,24118.0,2481.0,0.661414,0.406361,51959.0,43.0,0.954427,0.682979,0.709789,0.0,0.796203
9,0,UNet07_res512_23_12_23,0.45,26022.0,2481.0,0.637177,0.389396,50055.0,43.0,0.952775,0.657952,0.709789,0.0,0.778382


In [ ]:
ious = np.arange(0.5, 1, 0.05)
list_df_PR_iou_plot = []
for iou in ious:
  df_temp = df_PR_iou[df_PR_iou['IoU'] >=  iou]
  list_df_PR_iou_plot.append(
      pd.melt(
          df_temp,
          id_vars=['seuil', 'modele'],
          value_vars=['recall', 'precision', 'F1']).assign(seuil_iou = iou).groupby(['modele', 'seuil_iou', 'seuil', 'metrique']).mean().reset_index()

  )
df_plot = pd.concat(list_df_PR_iou_plot).reset_index()


In [ ]:
fig_iou = px.line(
    df_plot,
    x = 'seuil_iou',
    y = 'value',
    color = 'seuil',
    facet_col = 'metrique',
    facet_row = 'modele',
    hover_data=['seuil_iou', 'value'])
fig_iou.update_yaxes(matches = None)
fig_iou.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels = True, rangemode = 'tozero'))
fig_iou.update_layout(
    title = 'Meilleures courbes IoU',
    plot_bgcolor='white',
    height = 600,
    width = 900)
fig_iou.show()

## Instance segmentation

Calcul de la courbe PR:

+ recall : TP/(TP+FN)
+ precision : TP/(TP+FP)

*Le recall vaut 1 si FN = 0 (on a prédit tous les TP)*

*La precision vaut 1 si FP = 0 (pas de résultat faux)*

F1-score

F1 = (2.P.R)/(P+R)

In [ ]:
def calc_recall(TP, FN):
  if FN == 0:
    recall = 1
  else:
    recall = TP/(TP+FN)
  return recall

def calc_precision(TP, FP):
  if FP == 0:
    precision = 1
  else:
    precision = TP/(TP+FP)
  return precision


In [ ]:
color_discrete_map = {
    'UNet07_res512_23_12_23' : '#c994c7',
    'DeeplabV3Plus_3_20240112' : '#abd9e9',
    'SegFormer' : '#de77ae',
    'YOLOv8-seg' : '#dfc27d',
    'YOLOv8-seg (semantic)' : '#ff7f00',
    'YOLOv8-seg (Bretagne)' : '#80cdc1',
    'YOLOv8-seg-réentraîné': '#bf812d',
    'YOLOv8-seg-réentraîné (Bretagne)': '#35978f'
    }


In [ ]:
list_files = [
    '/content/drive/MyDrive/DataScientest/Keras/Moez-UNet07_res512_23_12_23_metriques_seuil_iou.csv',
    '/content/drive/MyDrive/DataScientest/Keras/DeeplabV3Plus_3_metriques_seuil_iou_20240112.csv',
    '/content/drive/MyDrive/DataScientest/Keras/SegFormer_metriques_seuil_iou_20240118.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_metriques_seuil_iou_20240124.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_metriques_seuil_iou_20240124-alt.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_hardcases_metriques_seuil_iou_20240124.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_hardcases_metriques_seuil_iou_20240124-alt.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_flat_metriques_seuil_iou_20240415.csv'
]
list_modeles = [
    'UNet07_res512_23_12_23',
    'DeeplabV3Plus_3_20240112',
    'SegFormer',
    'YOLOv8-seg',
    'YOLOv8-seg (Bretagne)',
    'YOLOv8-seg-réentraîné',
    'YOLOv8-seg-réentraîné (Bretagne)',
    'YOLOv8-seg (semantic)'
]
list_modeles_keep = [
    'UNet07_res512_23_12_23',
    'YOLOv8-seg',
    'YOLOv8-seg (semantic)'
]

list_df_PR_iou = []
for file, modele in zip(list_files, list_modeles):
  if modele in list_modeles_keep:
    df = pd.read_csv(file)
    df["seuil"] = df["seuil"].astype(float)
    df["seuil_iou"] = df["seuil_iou"].astype(float)
    df = df.groupby(['seuil', 'seuil_iou']).sum().reset_index()
    df['recall'] = df.apply(lambda row: calc_recall(row['TP'], row['FN']), axis = 1)
    df['precision'] = df.apply(lambda row: calc_precision(row['TP'], row['FP']), axis = 1)
    list_df_PR_iou.append(
        df.assign(modele = modele))


In [ ]:
palette = ['#fcbba1','#fc9272','#fb6a4a','#ef3b2c','#cb181d','#a50f15','#67000d']
fig_PR_iou = px.line(
    pd.concat(list_df_PR_iou),
    x = 'recall',
    y = 'precision',
    color = 'seuil_iou',
    facet_col = 'modele',
    hover_data=['seuil'],
    color_discrete_sequence = palette)
fig_PR_iou.update_layout(
    legend = dict(yanchor = 'bottom', y = -0.7, xanchor = 'center', x = 0),
    title = 'Courbe PR/IoU',
    plot_bgcolor='white',
    height = 600,
    width = 900)
fig_PR_iou.write_html('/content/drive/MyDrive/DataScientest/Keras/PR_seuils_iou.html')
fig_PR_iou.show()

In [ ]:
df_PR_iou = pd.concat(list_df_PR_iou).reset_index()
df_PR_iou['distance_ideal'] = np.sqrt((1 - df_PR_iou['precision'])**2 + (1 - df_PR_iou['recall'])**2)
df_PR_iou['rank'] = 1 - df_PR_iou['distance_ideal']
fig = px.scatter(
    df_PR_iou.sort_values(['modele', 'distance_ideal']).groupby(['modele']).head(1).sort_values('rank', ascending = False),
    x='modele',y='rank', color = 'modele',
    color_discrete_map = color_discrete_map,
    hover_data=['precision', 'recall'],
    title="Meilleur couple (precison, recall) par modèle")
fig.update_traces(marker={'size': 15})
fig.update_layout(
    showlegend = False,
    legend=dict(title="Modèle"),
    yaxis_range = [0, 1],
    plot_bgcolor='white',
    height = 500,
    width = 600
)
fig.write_html('/content/drive/MyDrive/DataScientest/Keras/best_PR_seuils_iou.html')
fig.show()


In [ ]:
df_best = df_PR_iou.sort_values(['modele', 'distance_ideal']).groupby(['modele']).head(1).sort_values('distance_ideal', ascending = True)
df_best.head()

,index,seuil,seuil_iou,Unnamed: 0,i_images,TP,FP,FN,recall,precision,modele,formes,distance_ideal,rank
154,7,0.05,0.01,165275250,1124250,39621,2257,6169,0.865276,0.946105,YOLOv8-seg,NaN,0.145104,0.854896
301,7,0.05,0.01,165275250,1124250,13307,1193,6664,0.666316,0.917724,YOLOv8-seg (semantic),30731.0,0.343677,0.656323
49,49,0.35,0.01,165338250,1124250,12401,803,6497,0.656207,0.939185,UNet07_res512_23_12_23,NaN,0.349130,0.650870


In [ ]:
df_best = df_PR_iou[df_PR_iou['seuil_iou']>=0.5].sort_values(['modele', 'distance_ideal']).groupby(['modele']).head(1).sort_values('distance_ideal', ascending = True)
df_best.head()

,index,seuil,seuil_iou,Unnamed: 0,i_images,TP,FP,FN,recall,precision,modele,formes,distance_ideal,rank
325,31,0.20,0.5,165311250,1124250,12232,2069,19941,0.380195,0.855325,YOLOv8-seg (semantic),30731.0,0.636467,0.363533
94,94,0.65,0.5,165405750,1124250,11328,1518,21163,0.348650,0.881831,UNet07_res512_23_12_23,NaN,0.661982,0.338018
199,52,0.35,0.5,165342750,1124250,11991,4423,19043,0.386383,0.730535,YOLOv8-seg,NaN,0.670177,0.329823


In [ ]:
df_best_iou = pd.concat(list_df_PR_iou)
df_best_iou = df_best_iou[df_best_iou['seuil_iou'] == 0.01]
fig_PR_iou = px.line(
    df_best_iou,
    x = 'recall',
    y = 'precision',
    color = 'modele',
    color_discrete_map = color_discrete_map,
    hover_data=['seuil'],
    color_discrete_sequence = palette)
fig_PR_iou.update_layout(
    title = 'Meilleures courbes PR/IoU',
    plot_bgcolor='white',
    height = 700,
    width = 700)
fig_PR_iou.write_html('/content/drive/MyDrive/DataScientest/Keras/PR_seuils_iou_best.html')
fig_PR_iou.show()

### Average metrics

In [ ]:
list_files = [
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_benchmark_metriques_seuil_iou_20240414.csv',
    '/content/drive/MyDrive/DataScientest/Keras/YOLOv8_flat_benchmark_metriques_seuil_iou_20240414.csv',
    '/content/drive/MyDrive/DataScientest/Keras/Moez-UNet07_res512_23_12_23_benchmark_metriques_seuil_iou_20240414.csv'
    ]
list_modeles = [
    'YOLOv8-seg',
    'YOLOv8-seg (semantic)',
    'UNet07_res512_23_12_23']
list_seuils = [
    0.35,
    0.20,
    0.65
]
seuils_formes = [0, 50, 3000]

list_df_average_metrics = []

for file, modele, seuil in zip(list_files, list_modeles, list_seuils):
  df = pd.read_csv(file)
  df = df[df['seuil'] == seuil]
  df['recall'] = df.apply(lambda row: calc_recall(row['TP'], row['FN']), axis = 1)
  df['precision'] = df.apply(lambda row: calc_precision(row['TP'], row['FP']), axis = 1)
  df['F1'] = df.apply(lambda row: calc_F1(row['precision'], row['recall']), axis = 1)
  list_df_average_metrics.append(df.assign(modele = modele))

df_average_metrics = pd.concat(list_df_average_metrics).reset_index()
df_average_metrics['formes_cat'] = pd.cut(df_average_metrics['formes'], seuils_formes, right = False, include_lowest = True)
df_average_metrics.head()

,index,Unnamed: 0,seuil,seuil_iou,i_images,TP,FP,FN,formes,recall,precision,F1,modele,formes_cat
0,70,70,0.35,0.50,0,17,4,26,43,0.395349,0.809524,0.531250,YOLOv8-seg,"[0, 50)"
1,71,71,0.35,0.55,0,17,4,27,43,0.386364,0.809524,0.523077,YOLOv8-seg,"[0, 50)"
2,72,72,0.35,0.60,0,15,6,28,43,0.348837,0.714286,0.468750,YOLOv8-seg,"[0, 50)"
3,73,73,0.35,0.65,0,13,8,30,43,0.302326,0.619048,0.406250,YOLOv8-seg,"[0, 50)"
4,74,74,0.35,0.70,0,11,10,33,43,0.250000,0.523810,0.338462,YOLOv8-seg,"[0, 50)"


In [ ]:
df_plot = pd.melt(
    df_average_metrics,
    id_vars=['modele','seuil_iou'], value_vars=['recall', 'precision', 'F1']).groupby(['modele','seuil_iou', 'variable']).mean().reset_index()

fig_iou = px.line(
    df_plot[df_plot['modele'].isin(['YOLOv8-seg', 'UNet07_res512_23_12_23'])],
    x = 'seuil_iou',
    y = 'value',
    color = 'modele',
    color_discrete_map = color_discrete_map,
    facet_col = 'variable',
    hover_data=['seuil_iou', 'value'])
fig_iou.update_yaxes(matches = None)
fig_iou.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels = True, rangemode = 'tozero'))
fig_iou.update_layout(
    legend = dict(yanchor = 'bottom', y = -0.5, xanchor = 'center', x = 0),
    title = '',
    plot_bgcolor='white',
    height = 400,
    width = 900)
fig_iou.show()

In [ ]:
fig_iou = px.line(
    df_plot[df_plot['modele'].isin(['UNet07_res512_23_12_23', 'YOLOv8-seg (semantic)'])],
    x = 'seuil_iou',
    y = 'value',
    color = 'modele',
    color_discrete_map = color_discrete_map,
    facet_col = 'variable',
    hover_data=['seuil_iou', 'value'])
fig_iou.update_yaxes(matches = None)
fig_iou.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels = True, rangemode = 'tozero'))
fig_iou.update_layout(
    legend = dict(yanchor = 'bottom', y = -0.5, xanchor = 'center', x = 0),
    title = '',
    plot_bgcolor='white',
    height = 400,
    width = 900)
fig_iou.show()

In [ ]:
fig_iou = px.line(
    df_plot[df_plot['modele'].isin(['YOLOv8-seg', 'YOLOv8-seg (semantic)'])],
    x = 'seuil_iou',
    y = 'value',
    color = 'modele',
    color_discrete_map = color_discrete_map,
    facet_col = 'variable',
    hover_data=['seuil_iou', 'value'])
fig_iou.update_yaxes(matches = None)
fig_iou.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels = True, rangemode = 'tozero'))
fig_iou.update_layout(
    legend = dict(yanchor = 'bottom', y = -0.5, xanchor = 'center', x = 0),
    title = '',
    plot_bgcolor='white',
    height = 400,
    width = 900)
fig_iou.show()

In [ ]:
df_plot[df_plot['seuil_iou']>=0.5][['modele', 'variable', 'value']].groupby(['modele', 'variable']).mean().reset_index().pivot_table(
    values='value', index=['modele'], columns='variable', aggfunc="sum").reset_index()


variable,modele,F1,precision,recall
0,UNet07_res512_23_12_23,0.427227,0.607361,0.390289
1,YOLOv8-seg,0.384266,0.588083,0.354062
2,YOLOv8-seg (semantic),0.390651,0.543124,0.371764


In [ ]:
df_plot[['modele', 'seuil_iou', 'variable', 'value']].groupby(['modele', 'seuil_iou','variable']).mean().reset_index().pivot_table(
    values='value', index=['modele', 'seuil_iou'], columns='variable', aggfunc="sum").reset_index()


variable,modele,seuil_iou,F1,precision,recall
0,UNet07_res512_23_12_23,0.50,0.604907,0.889714,0.529005
1,UNet07_res512_23_12_23,0.55,0.586883,0.863685,0.514438
2,UNet07_res512_23_12_23,0.60,0.562443,0.826266,0.495106
3,UNet07_res512_23_12_23,0.65,0.523462,0.765802,0.463833
4,UNet07_res512_23_12_23,0.70,0.471630,0.680551,0.424159
5,UNet07_res512_23_12_23,0.75,0.403716,0.568820,0.371798
6,UNet07_res512_23_12_23,0.80,0.340303,0.464910,0.323461
7,UNet07_res512_23_12_23,0.85,0.286279,0.378283,0.281766
8,UNet07_res512_23_12_23,0.90,0.252683,0.326790,0.254877
9,UNet07_res512_23_12_23,0.95,0.239970,0.308793,0.244445


In [ ]:
df_plot_formes = pd.melt(
    df_average_metrics,
    id_vars=['modele','seuil_iou', 'formes_cat'], value_vars=['recall', 'precision', 'F1']).groupby(['modele','seuil_iou', 'formes_cat', 'variable']).mean().reset_index()

fig_iou_formes = px.line(
    df_plot_formes,
    x = 'seuil_iou',
    y = 'value',
    color = 'modele',
    color_discrete_map = color_discrete_map,
    facet_col = 'variable',
    facet_row = 'formes_cat',
    hover_data=['seuil_iou', 'value'])
fig_iou_formes.update_yaxes(matches = None)
fig_iou_formes.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels = True, rangemode = 'tozero'))
fig_iou_formes.update_layout(
    legend = dict(yanchor = 'bottom', y = -0.5, xanchor = 'center', x = 0),
    title = '',
    plot_bgcolor='white',
    height = 900,
    width = 900)
fig_iou_formes.show()

In [ ]:
df_plot_formes[df_plot_formes['seuil_iou']>=0.5][['modele', 'variable', 'formes_cat', 'value']].groupby(['modele', 'formes_cat', 'variable']).mean().reset_index().pivot_table(
    values='value', index=['modele', 'formes_cat'], columns='variable', aggfunc="sum").reset_index()


variable,modele,formes_cat,F1,precision,recall
0,UNet07_res512_23_12_23,"[0, 50)",0.461289,0.635035,0.430563
1,UNet07_res512_23_12_23,"[50, 3000)",0.237200,0.452970,0.165600
2,YOLOv8-seg,"[0, 50)",0.421564,0.632881,0.395686
3,YOLOv8-seg,"[50, 3000)",0.176178,0.338160,0.121843
4,YOLOv8-seg (semantic),"[0, 50)",0.425051,0.577721,0.412914
5,YOLOv8-seg (semantic),"[50, 3000)",0.198736,0.350109,0.142187
